<a href="https://colab.research.google.com/github/sayakpaul/Reproducibility-in-tf.keras-with-wandb/blob/master/Part_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Set up deterministic flag
import tensorflow as tf
import os
os.environ["TF_DETERMINISTIC_OPS"] = "1"

In [0]:
# Fix random seeds
SEED = 666
tf.random.set_seed(SEED)
import numpy as np
np.random.seed(SEED)

In [0]:
# Import wandb
import wandb
from wandb.keras import WandbCallback

In [0]:
# Other imports
import time

In [0]:
# Load up and preprocess data
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

In [0]:
# Verify shapes
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [0]:
# Specify the class labels
LABELS = ["T-shirt/top","Trouser","Pullover","Dress","Coat",
        "Sandal","Shirt","Sneaker","Bag","Ankle boot"]

In [0]:
# Create a sepcific validation set from the test set
X_val = []
y_val = []

idx = np.random.choice(test_images.shape[0], 32)
for i in idx:
    X_val.append(test_images[i])
    y_val.append(test_labels[i])
    
X_val, y_val = np.array(X_val), np.array(y_val)

In [0]:
# Verify shapes
X_val.shape, y_val.shape

((32, 28, 28), (32,))

In [0]:
# Define model configurations in a dictionary
config_defaults = {
        "epochs": 10,
        "batch_size": 128,
        "prefinal_activation": "relu",
        "final_activation": "softmax",
        "optimizer": "adam",
        'seed': 42
}

In [0]:
# Initialize a new wandb run
wandb.init(project="reproducible-ml", id="no-sweeps", config=config_defaults)

# Config is a variable that holds and saves hyperparameters and inputs
config = wandb.config

Error generating diff: Reference at 'refs/remotes/origin/master' does not exist


In [0]:
# Set up early stopping callback
es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                      patience=1,
                                      restore_best_weights=True,
                                      verbose=3)

In [0]:
# Utility function for model
def get_training_model():
    # Define the model
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(256, activation=config.prefinal_activation),
        tf.keras.layers.Dense(10, activation=config.final_activation)
    ])
    
    # Compile the model
    model.compile(optimizer=config.optimizer,
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model

In [0]:
# Train the model
start = time.time()
mlp_model = get_training_model()
mlp_model.fit(train_images, train_labels, 
    validation_data=(test_images, test_labels),
    batch_size=config.batch_size,
    epochs=config.epochs,
    callbacks=[WandbCallback(data_type="image", validation_data=(X_val, y_val), 
                    labels=LABELS),
              es])
wandb.log({"training_time":time.time()-start})

Error generating diff: Reference at 'refs/remotes/origin/master' does not exist


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 45us/sample - loss: 0.5299 - accuracy: 0.8147 - val_loss: 0.4510 - val_accuracy: 0.8429
Epoch 2/10
60000/60000 [==============================] - 1s 24us/sample - loss: 0.3899 - accuracy: 0.8612 - val_loss: 0.4069 - val_accuracy: 0.8550
Epoch 3/10
60000/60000 [==============================] - 1s 24us/sample - loss: 0.3474 - accuracy: 0.8751 - val_loss: 0.3722 - val_accuracy: 0.8629
Epoch 4/10
60000/60000 [==============================] - 1s 24us/sample - loss: 0.3189 - accuracy: 0.8850 - val_loss: 0.3599 - val_accuracy: 0.8709
Epoch 5/10
60000/60000 [==============================] - 1s 24us/sample - loss: 0.3039 - accuracy: 0.8880 - val_loss: 0.3498 - val_accuracy: 0.8755
Epoch 6/10
60000/60000 [==============================] - 1s 24us/sample - loss: 0.2871 - accuracy: 0.8940 - val_loss: 0.3409 - val_accuracy: 0.8783
Epoch 7/10
60000/60000 [==============================] 

In [0]:
# Running the model a few times locally
accuracy_list = []
loss_list = []
times = []

for i in range(15):
    print("Running run number ",i+1)
    start = time.time()
    mlp_model = get_training_model()
    history = mlp_model.fit(train_images, train_labels, 
        validation_data=(test_images, test_labels),
        batch_size=config.batch_size,
        epochs=config.epochs,
        callbacks=[es],
        verbose=0)
    end = time.time() - start
    
    val_accuracy = history.history["val_accuracy"][-1]
    val_loss = history.history["val_loss"][-1]
    
    accuracy_list.append(val_accuracy)
    loss_list.append(val_loss)
    times.append(end)
    
print("Mean val accuracy {} mean val loss {}".format(np.mean(accuracy_list), np.mean(loss_list)))
print("Std val accuracy {} Std val loss {}".format(np.std(accuracy_list), np.std(loss_list)))
print("Mean time to train {}".format(np.mean(times)))
print("Std time to train {}".format(np.std(times)))

Running run number  1
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Running run number  2
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Running run number  3
Restoring model weights from the end of the best epoch.
Epoch 00006: early stopping
Running run number  4
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Running run number  5
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Running run number  6
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Running run number  7
Restoring model weights from the end of the best epoch.
Epoch 00006: early stopping
Running run number  8
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Running run number  9
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Running run number  10
Restoring model weights

We can see that the standard deviation of the validation accuracy and validation loss are pretty low. So, we are good here. You might thnik that the dataset and the model are too low to conclude anything but note that as you would scale up these concepts would still apply there :)